<h3><B>conda environment:</b> pertpy_env, python v3.11.14

In [ ]:
from filter_adata import adata_filtered, adata_hightumour, adata_peritumour

import importlib
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns

import pertpy as pt
from pathlib import Path 

home_path = '/Users/nabilazulkapeli/Documents/Honours Thesis 2025/nabs_data'
home_path = Path(home_path)
CTP_path = home_path / f'CTP_figures'
CTP_path.mkdir(parents=True, exist_ok=True)
CTP_path = Path(CTP_path)

sns.set_theme(context='paper', style='ticks',font='sans-serif')

In [ ]:
# dictionaries for plots

simple_dict = {
    "CD8 T": "Immune",
    "CD4 T": "Immune",
    "Plasma": "Immune",
    "Plasmablast": "Immune",
    "TLS": "Immune",
    "Dendritic": "Immune",
    "Granulocyte": "Immune",
    "M1 TAM": "Immune",
    "M2 TAM": "Immune",
    "Ig-TAM": "Immune",
    "iCAF": "Stromal",
    "cCAF": "Stromal",
    "Endothelial": "Stromal",
    "Epithelial": "Stromal",
    "Melanoma": "Tumour",
    "Mast": "Immune"
}

response_palette = {"Responder": "#66c2a5", "Non-Responder": "#fc8d62"}

response_order = [
    "Responder",
    "Non-Responder"
]

new_labels_colors = {
    "Melanoma": "#b0b0b0",
    "Endothelial": "#7C3C67",
    "Epithelial": "#AF5A93",
    "cCAF": "#BD75A5",
    "iCAF": "#CA91B7",
    "Mast": "#4B3D80",
    "Granulocyte": "#67589B",
    "Dendritic": "#264566",
    "M1 TAM": "#496F97",
    "M2 TAM": "#5B83AE",
    "Ig-TAM": "#7697BC",
    "Plasmablast": "#3E7682",
    "Plasma": "#4D99A8",
    "TLS": "#65AAB8",
    "CD4 T": "#81BAC5",
    "CD8 T": "#ABD1D8"
}

core_order = [
'High Tumour: 0040207_Region_4_14-D'
 'High Tumour: 0029039_Region_1_4-C'
 'High Tumour: 0029039_Region_1_4-E'
 'High Tumour: 0029039_Region_1_5-A'
 'High Tumour: 0029039_Region_1_5-C'
 'High Tumour: 0029039_Region_1_5-E'
 'High Tumour: 0029039_Region_1_6-A'
 'High Tumour: 0029039_Region_1_6-C'
 'High Tumour: 0029039_Region_1_6-E'
 'High Tumour: 0029039_Region_2_10-E'
 'High Tumour: 0029039_Region_2_11-E'
 'High Tumour: 0029039_Region_2_7-C'
 'High Tumour: 0029039_Region_2_7-E'
 'High Tumour: 0029039_Region_2_8-E'
 'High Tumour: 0029039_Region_2_9-E'
 'High Tumour: 0029039_Region_3_10-A'
 'High Tumour: 0029039_Region_3_10-C'
 'High Tumour: 0029039_Region_3_11-A'
 'High Tumour: 0029039_Region_3_7-A'
 'High Tumour: 0029039_Region_3_8-A'
 'High Tumour: 0029039_Region_3_8-C'
 'High Tumour: 0029039_Region_3_9-A'
 'High Tumour: 0029039_Region_3_9-C'
 'High Tumour: 0040207_Region_4_14-A'
 'High Tumour: 0040207_Region_4_14-B'
 'High Tumour: 0040207_Region_4_14-C'
 'Peritumour: 0029039_Region_1_4-B'
 'Peritumour: 0029039_Region_1_4-D'
 'Peritumour: 0029039_Region_1_4-F'
 'Peritumour: 0029039_Region_1_5-B'
 'Peritumour: 0029039_Region_1_6-B'
 'Peritumour: 0029039_Region_1_6-D'
 'Peritumour: 0029039_Region_1_6-F'
 'Peritumour: 0029039_Region_2_10-D'
 'Peritumour: 0029039_Region_2_7-F'
 'Peritumour: 0029039_Region_2_8-F'
 'Peritumour: 0029039_Region_2_9-D'
 'Peritumour: 0029039_Region_2_9-F'
 'Peritumour: 0029039_Region_3_10-B'
 'Peritumour: 0029039_Region_3_11-B'
 'Peritumour: 0029039_Region_3_7-B'
 'Peritumour: 0029039_Region_3_8-B'
 'Peritumour: 0029039_Region_3_9-B'
 'Peritumour: 0052306_Region_4_14-A'
 'Peritumour: 0052306_Region_4_14-D'
]

In [ ]:
# make a copy!
response_subset = adata_filtered.copy()

<h1><b>Broad Cell Type Deconvolution

In [ ]:
# only run on global
broad_model = pt.tl.Sccoda()
broad_data = broad_model.load(
    response_subset,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="new_broad_labels",
    sample_identifier="core_id",
    covariate_obs=["Response"],
)

In [ ]:
broad_data.mod["global"] = broad_data["coda"][
    broad_data["coda"].obs["Response"].isin(["Responder", "Non-Responder"])
].copy()
print(broad_data["global"])

In [ ]:
broad_model.plot_boxplots(broad_data, modality_key="global", feature_name="Response", figsize=(20,10), add_dots=False, palette=response_palette, return_fig=True)
plt.savefig(CTP_path / 'broad_global_box.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
broad_model.plot_stacked_barplot(broad_data, modality_key="global", feature_name="core_id",figsize=(15,6), return_fig=True)
plt.savefig(CTP_path / 'broad_global_core_variation.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
broad_model.plot_stacked_barplot(broad_data, modality_key="global", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'broad_global_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
broad_data = broad_model.prepare(
    broad_data,
    modality_key="global",
    formula="Response",
    reference_cell_type="automatic",
)
broad_model.run_nuts(broad_data, modality_key="global", rng_key=1234)

In [ ]:
broad_model.summary(broad_data, modality_key="global")

<h1><b>Specific Cell Type Deconvolution

<h2><B>1. Global

In [ ]:
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    response_subset,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="new_specific_labels",
    sample_identifier="core_id",
    covariate_obs=["Response"],
)

In [ ]:
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
sccoda_data.mod["global"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["Response"].isin(["Responder", "Non-Responder"])
].copy()
print(sccoda_data["global"])

In [ ]:
sccoda_model.plot_boxplots(sccoda_data, modality_key="global", feature_name="Response", figsize=(20,10), add_dots=False, palette=response_palette, return_fig=True)
plt.savefig(CTP_path / 'global_box.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="global", feature_name="core_id",figsize=(15,6), return_fig=True)
plt.savefig(CTP_path / 'global_core_variation.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="global", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'global_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="global",
    formula="Response",
    reference_cell_type="automatic",
)
sccoda_model.run_nuts(sccoda_data, modality_key="global", rng_key=1234)

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="global", est_fdr=0.2)

In [ ]:
sccoda_model.summary(sccoda_data, modality_key="global")

In [ ]:
sccoda_model.plot_effects_barplot(sccoda_data, modality_key="global", parameter="Final Parameter")

In [ ]:
sccoda_data["global"]

In [ ]:
sccoda_data["global"].varm["effect_df_Response[T.Responder]"]

In [ ]:
global_model = sccoda_data["global"].varm["effect_df_Response[T.Responder]"]
global_model.to_excel(CTP_path / 'global_results.xlsx', index=True)

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="global")

In [ ]:
sccoda_model.plot_rel_abundance_dispersion_plot(sccoda_data, modality_key="global", abundant_threshold=0.9)

<h2><b>2. High-Tumour

In [ ]:
broad_data.mod["ht"] = broad_data["coda"][
    broad_data["coda"].obs["core_id"].str.contains("High Tumour")
].copy()
print(broad_data["ht"])

In [ ]:
broad_model.plot_stacked_barplot(broad_data, modality_key="ht", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'ht_broad_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_data.mod["ht"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["core_id"].str.contains("High Tumour")
].copy()
print(sccoda_data["ht"])

In [ ]:
sccoda_data

In [ ]:
sccoda_model.plot_boxplots(sccoda_data, modality_key="ht", feature_name="Response", figsize=(20,10), add_dots=False, palette=response_palette, return_fig=True)
plt.savefig(CTP_path / 'ht_box.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="ht", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'ht_specific_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="ht",
    formula="Response",
    reference_cell_type="automatic",
)
sccoda_model.run_nuts(sccoda_data, modality_key="ht", rng_key=1234)

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="ht", est_fdr=0.2)

In [ ]:
sccoda_model.summary(sccoda_data, modality_key="ht")

In [ ]:
sccoda_model.plot_rel_abundance_dispersion_plot(sccoda_data, modality_key="ht", abundant_threshold=0.9)

In [ ]:
ht_model = sccoda_data["ht"].varm["effect_df_Response[T.Responder]"]
ht_model.to_excel(CTP_path / 'ht_results.xlsx', index=True)

In [ ]:
sccoda_model.plot_effects_barplot(sccoda_data, modality_key="ht", parameter="Final Parameter", return_fig=True)
plt.savefig(CTP_path / "ht_significant_fdr_03.pdf", dpi=300, bbox_inches='tight')

ht_model = sccoda_data["ht"].varm["effect_df_Response[T.Responder]"]
ht_model.to_excel(CTP_path / 'ht_results.xlsx', index=True)

sccoda_model.credible_effects(sccoda_data, modality_key="ht")

<h2><b>3. Peritumour

In [ ]:
broad_data.mod["peri"] = broad_data["coda"][
    broad_data["coda"].obs["core_id"].str.contains("Peritumour")
].copy()
print(broad_data["peri"])

In [ ]:
broad_model.plot_stacked_barplot(broad_data, modality_key="peri", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'peri_broad_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_data.mod["peri"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["core_id"].str.contains("Peritumour")
].copy()
print(sccoda_data["peri"])

In [ ]:
sccoda_model.plot_boxplots(sccoda_data, modality_key="peri", feature_name="Response", figsize=(20,10), add_dots=False, palette=response_palette, return_fig=True)
plt.savefig(CTP_path / 'peri_box.pdf', dpi=300, bbox_inches='tight')

In [ ]:
sccoda_model.plot_stacked_barplot(sccoda_data, modality_key="peri", feature_name="Response", figsize=(8,8), level_order=response_order, return_fig=True)
plt.savefig(CTP_path / 'peri_specific_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="peri",
    formula="Response",
    reference_cell_type="automatic",
)
sccoda_model.run_nuts(sccoda_data, modality_key="peri", rng_key=1234)

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="peri", est_fdr=0.2)

In [ ]:
sccoda_model.summary(sccoda_data, modality_key='peri')

In [ ]:
sccoda_model.plot_effects_barplot(sccoda_data, modality_key="peri", parameter="Final Parameter", return_fig=True)
plt.savefig(CTP_path / "peri_significant_fdr_02.pdf", dpi=300, bbox_inches='tight')

ht_model = sccoda_data["peri"].varm["effect_df_Response[T.Responder]"]
ht_model.to_excel(CTP_path / 'peri_results.xlsx', index=True)

sccoda_model.credible_effects(sccoda_data, modality_key="peri")